### Karhuhen-Loève Monte Carlo (KLMC)

Consider  payoffs of the form $$\varphi_m(X_{\tau}) = (h_m \circ  f)(X_{\tau}),$$
where $f$ is a running functional and $h_m:\mathbb{R} \to \mathbb{R}$ that depends on some parameter $m \in \mathbb{R} $. 


Example: A call option on $Y=f(X)$  is obtained with $h_m(y) = (y-m\, x_0)^{+}$ and $m$ is the _moneyness_ of the claim. 

In [1]:
import numpy as np
import numpy.random as rdm
from scipy import interpolate as it
from numpy import linalg as ln

from ttictoc import tic,toc
import matplotlib.pyplot as plt; plt.style.use('dark_background')
import matplotlib.colors as colors
from matplotlib import cm
__ = np.newaxis

import pandas as pd

In [2]:
# Parameters
T, Noff = 1, int(1e3)
ts, dt = np.linspace(0,T,Noff+1), T/Noff
Ts     = ts[:,__]
# Parameters
x0, r, delta, sig = 100, 0.0, 0.0, 0.2
# Ito map (Black-Scholes model)
def w2X(W,Ts): return x0 * np.exp((r-delta-sig**2/2)*Ts + sig*W)
# Functionals 
f = {"Asian"   : lambda X: np.vstack([X[0,:],np.cumsum((X[:-1,:] + X[1:,:])/2,axis=0)\
                          /np.arange(1,np.shape(X)[0])[:,__]]),
     "Lookback": lambda X: np.maximum.accumulate(X,axis = 0)}
# Vanilla payoffs
hV  = lambda y,m: np.maximum(y-m*x0,0.)
# Up & Out Digital (same underlying functional as the lookback)
f["U&O"] = f["Lookback"]
# Up & Out digital payoff  
hUO = lambda y,b: 1 * (y < b*x0)
# Payoffs
h  = {"Asian": hV, "Lookback": hV, "U&O": hUO}               
# Truncation Levels
K  = {"Asian": 40, "Lookback": 100, "U&O": 100} 
# Number of simulations (offline and online)
Joff,J = 2**17, 2**19 
# Moneyness/Barrier Level
Ms = {"Asian"   : np.linspace(0.75,1.25,11), 
      "Lookback": np.linspace(0.75,1.25,11),
      "U&O"     : np.linspace(1.05,1.5,10)}
# Maturity Grid
nT = 52; Taus = np.linspace(1/nT,T,nT)

### Karhunen-Loève Transform

In [3]:
def BM(T,N,nSim): 
    """Brownian Motions"""
    # Brownian increments
    dW = np.sqrt(T/N) * rdm.randn(N,nSim)
    return np.vstack([np.zeros_like(dW[0,:]),np.cumsum(dW,axis=0)])

# Trapezoidal Weights
w = 2**(np.sum(np.eye(Noff+1)[:,1:-1],axis=1,keepdims = True) -1)
# L^2 Norm
L2Norm = lambda Y: np.sqrt(np.sum(w * Y**2,axis = 0) * dt)

def KL(kappa):
    """Karhunen-Loève Basis"""
    # Solve the Fredholm integral equations 
    # (with trapezoidal rule) in one go
    lbda, F = ln.eig(kappa * w.T * dt) 
    return F / L2Norm(F), lbda

# Thin partition of [0,1] for the quantile function
us = np.linspace(0,1,int(1e4)) 

def klmcOff(f,K):
    """KLMC Method, Offline Phase"""
    # Simulate trajectories of f(X)
    Y =  f(w2X(BM(T,Noff,Joff),Ts))
    # Mean function
    mu_ = np.mean(Y,axis=1,keepdims = True)
    # Eigendecomposition
    F_,lbda = KL(np.cov(Y))
    # L2 coefficients
    xi = F_[:,:K].T @ (w * (Y - mu_)) * dt
    # Interpolated mean and eigenfunctions (to price options for any maturity)
    mu = it.interp1d(ts,mu_.flatten()) 
    F  = [it.interp1d(ts,F_[:,k]) for k in range(K)]
    # Quantile Function
    phiInv = [it.interp1d(us,np.quantile(xi[k,:],us)) for k in range(K)]
    return F,mu,phiInv

In [4]:
# Offline Phase
F,mu,phiInv = {},{},{}
for key in ["Asian","Lookback"]: F[key],mu[key],phiInv[key] = klmcOff(f[key],K[key])
# Use the Lookback output for Up and Out Digital options
F["U&O"],mu["U&O"],phiInv["U&O"] = F[key],mu[key],phiInv[key]

### Price Surface

Standard Monte Carlo (MC)
$$
p^{N,J}(m,\tau) = \frac{1}{J}\sum_{j=1}^J \varphi_m(X^{N,j}_\tau).
$$



Karhunen-Loève Monte Carlo (KLMC)
$$
p^{K,\mathfrak{F},J}(m,\tau) = \frac{1}{J}\sum_{j=1}^J h_m(y_\tau^{K,\mathfrak{F},j}).
$$

In [5]:
def mcPrice(Y,h,Ms,Taus):
    """Monte Carlo Price Surface"""
    return np.vstack([np.exp(-r*Taus)*np.mean(h(Y,m),axis=1) for m in Ms])

def klmcOn(phiInv,F,mu,h,J,Ms,Taus):
    """KLMC Method, Online Phase"""
    # 1. Simulate L^2 coefficients
    xi = np.vstack([ph(rdm.rand(J)) for ph in phiInv])
    # 2. Transformed paths
    Y  = mu(Taus)[:,__] + np.vstack([F_(Taus) for F_ in F]).T @ xi
    # 3. Monte Carlo price
    return mcPrice(Y,h,Ms,Taus)

def mc(f,h,J,N,ts,Ms,Taus):
    """Standard Monte Carlo (MC)"""
    # Simulate underlying stock price
    X   = w2X(BM(T,N,J),ts[:,__])
    # Locate maturities
    ids = [np.where(np.round(ts-tau,12) == 0)[0][0] for tau in Taus]
    # Price Surface
    return mcPrice(f(X)[ids],h,Ms,Taus)

In [6]:
# Create directories to store the results
!mkdir KLMC MC Results

mkdir: KLMC: File exists
mkdir: MC: File exists
mkdir: Results: File exists


In [7]:
def priceSurf(met,nMC = 0,name = ""): 
    # Price Surface
    p,runTime = {},{}
    for key in h.keys(): 
        tic()   
        if met == "KLMC":
            p[key] = klmcOn(phiInv[key],F[key],mu[key],h[key],J,Ms[key],Taus)  
            info = "K = %d"%K[key]
        else:
            ts_ = np.linspace(0,T,nMC[key]+1)
            p[key] = mc(f[key],h[key],J,nMC[key],ts_,Ms[key],Taus) 
            info = "N = %d"%nMC[key]
        runTime[key] = [toc()]   
        # Save results
        out = pd.DataFrame(p[key],columns = Taus,index = Ms[key])
        out.to_csv("%s/%s,J = %d,%s%s.csv"%(met,key,J,info,name)) 
    out2 = pd.DataFrame.from_dict(runTime)
    out2.to_csv("%s/Runtime,J = %d%s.csv"%(met,J,name))

In [8]:
# KLMC
priceSurf("KLMC")

In [9]:
# MC
nMC =  {"Asian": nT, "Lookback": 4*nT, "U&O": 4*nT}   
priceSurf("MC",nMC)

In [10]:
# Benchmark
nB =  {"Asian": 40*nT, "Lookback": 40*nT, "U&O": 40*nT} 
priceSurf("MC",nB,",Benchmark")

## Error

In [11]:
methods = ["KLMC","MC"]
info    = {"KLMC": {key: "K = %d"%K[key] for key in f.keys()},
           "MC"  : {key: "N = %d"%nMC[key] for key in f.keys()}}

p,err,mse = {m: {} for m in methods},{m: {} for m in methods},{m: {} for m in methods}
runtime = {}
for key in f.keys():
    pBench = pd.read_csv("MC/%s,J = %d,N = %d,Benchmark.csv"%(key,J,nB[key]),index_col=0).to_numpy()
    for met in methods:
        df = pd.read_csv("%s/%s,J = %d,%s.csv"%(met,key,J,info[met][key]),index_col=0)
        p[met][key]   = df.to_numpy()
        err[met][key] = p[met][key] - pBench
        mse[met][key] = np.round(np.mean(err[met][key]**2),5)
        runtime[met]  = pd.read_csv("%s/Runtime,J = %d.csv"%(met,J),index_col=0)
outMSE = pd.DataFrame.from_dict(mse)
outRT  = pd.concat([out for out in runtime.values()]).T; outRT.columns = ["KLMC time","MC time"]
outVal = pd.DataFrame.from_dict([K,nMC]).T            ; outVal.columns = ["K","N"]
out = pd.concat([outMSE,outRT,outVal],axis=1)[["K","KLMC","KLMC time","N","MC","MC time"]]
out.to_csv("Results/MSE and Runtime.csv")